In [16]:
!pip install mlflow boto3 awscli optuna imbalanced-learn lightgbm 

  Using cached botocore-1.35.99-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.10.4-py3-none-any.whl.metadata (1.7 kB)
INFO: pip is looking at multiple versions of awscli to determine which version is compatible with other requirements. This could take a while.
  Using cached awscli-1.44.38-py3-none-any.whl.metadata (11 kB)
  Using cached awscli-1.44.37-py3-none-any.whl.metadata (11 kB)
  Using cached awscli-1.44.36-py3-none-any.whl.metadata (11 kB)
  Using cached awscli-1.44.35-py3-none-any.whl.metadata (11 kB)
  Using cached awscli-1.44.34-py3-none-any.whl.metadata (11 kB)
  Using cached awscli-1.44.33-py3-none-any.whl.metadata (11 kB)
INFO: pip is still looking at multiple versions of awscli to determine which version is compatible with other requirements. This could take a while.
  Using cached awscli-1.44.32-py3-none-any.whl.metadata (11 kB)
  Using cached awscli-1.44.31-py3-none-any.whl.metadata (11 kB)
  Using cached awscli-1.44.30-py3-none-any.whl.metadata (11 k

In [17]:
!aws configure

AWS Access Key ID [****************BT4B]: ^C


In [19]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-184-73-117-95.compute-1.amazonaws.com:5000/")

In [20]:
# Set or create an experiment
mlflow.set_experiment("LightGBM HP Tuning")

2026/02/17 22:43:51 INFO mlflow.tracking.fluent: Experiment with name 'LightGBM HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-tracking-buc25/2', creation_time=1771348432165, experiment_id='2', last_update_time=1771348432165, lifecycle_stage='active', name='LightGBM HP Tuning', tags={}>

In [21]:
import pandas as pd

df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [22]:
!pip install mlflow 



In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

In [24]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

In [25]:
# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [26]:
# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [27]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run():
        # Log model type and trial number
        mlflow.set_tag("mlflow.runName", f"Trial_{trial_number}_{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Log hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

        return accuracy




In [28]:
# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    # Hyperparameter space to explore
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    num_leaves = trial.suggest_int('num_leaves', 20, 150)
    min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True)  # L1 regularization
    reg_lambda = trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True)  # L2 regularization

    # Log trial parameters
    params = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'num_leaves': num_leaves,
        'min_child_samples': min_child_samples,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda
    }

    # Create LightGBM model
    model = LGBMClassifier(n_estimators=n_estimators,
                           learning_rate=learning_rate,
                           max_depth=max_depth,
                           num_leaves=num_leaves,
                           min_child_samples=min_child_samples,
                           colsample_bytree=colsample_bytree,
                           subsample=subsample,
                           reg_alpha=reg_alpha,
                           reg_lambda=reg_lambda,
                           random_state=42)

    # Log each trial as a separate run in MLflow
    accuracy = log_mlflow("LightGBM", model, X_train, X_test, y_train, y_test, params, trial.number)

    return accuracy




In [29]:
# Step 7: Run Optuna for LightGBM, log the best model, and plot the importance of each parameter
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=100)  # Increased to 100 trials

    # Get the best parameters
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'],
                                learning_rate=best_params['learning_rate'],
                                max_depth=best_params['max_depth'],
                                num_leaves=best_params['num_leaves'],
                                min_child_samples=best_params['min_child_samples'],
                                colsample_bytree=best_params['colsample_bytree'],
                                subsample=best_params['subsample'],
                                reg_alpha=best_params['reg_alpha'],
                                reg_lambda=best_params['reg_lambda'],
                                random_state=42)

    # Log the best model with MLflow and print the classification report
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, best_params, "Best")

    # Plot parameter importance
    optuna.visualization.plot_param_importances(study).show()

    # Plot optimization history
    optuna.visualization.plot_optimization_history(study).show()

In [30]:
# Run the experiment for LightGBM
run_optuna_experiment()

[I 2026-02-17 22:44:07,745] A new study created in memory with name: no-name-fee2a7f5-388f-4de8-b44c-65d02f2bde95


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047727 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 98983
[LightGBM] [Info] Number of data points in the train set: 37848, number of used features: 967
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

/opt/miniconda3/envs/mlproj/lib/python3.11/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2026/02/17 22:44:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/miniconda3/envs/mlproj/lib/python3.11/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run Trial_0_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-184-73-117-95.compute-1.amazonaws.com:5000/#/experiments/2/runs/1eddd03ff3f34f059c49587315c4ec1a
🧪 View experiment at: http://ec2-184-73-117-95.compute-1.amazonaws.com:5000/#/experiments/2


[W 2026-02-17 22:45:19,613] Trial 0 failed with parameters: {'n_estimators': 468, 'learning_rate': 0.00010370795143774409, 'max_depth': 5, 'num_leaves': 22, 'min_child_samples': 32, 'colsample_bytree': 0.5700710957198979, 'subsample': 0.7634232017942243, 'reg_alpha': 1.0254694819383974, 'reg_lambda': 0.00030481321045237425} because of the following error: S3UploadFailedError('Failed to upload /var/folders/5m/v5yrtt052rn30tylrm068x680000gn/T/tmpx4uc0nda/model/python_env.yaml to mlflow-tracking-buc25/2/models/m-47ad00607aaf429e9eabcf82692355ee/artifacts/python_env.yaml: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied').
Traceback (most recent call last):
  File "/opt/miniconda3/envs/mlproj/lib/python3.11/site-packages/boto3/s3/transfer.py", line 452, in upload_file
  File "/opt/miniconda3/envs/mlproj/lib/python3.11/site-packages/s3transfer/futures.py", line 111, in result
    def set_exception(self, exception):
                   ^^^^^^^^^^^^^^^^^^^^^

S3UploadFailedError: Failed to upload /var/folders/5m/v5yrtt052rn30tylrm068x680000gn/T/tmpx4uc0nda/model/python_env.yaml to mlflow-tracking-buc25/2/models/m-47ad00607aaf429e9eabcf82692355ee/artifacts/python_env.yaml: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied

In [ ]:
best_model